In [ ]:
%matplotlib inline
import editdistance
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
from glob import glob

In [ ]:
landscapes=glob("data/TF_binding_landscapes/*")

In [ ]:
def is_sequence_a_peak(model,sequence,alphabet="AGTC"):
    neighbor=[s for s in sequence]
    sequence_fitness=model.get_fitness(sequence)
    for position in range(len(sequence)):
        for aa in alphabet:
            if aa!=sequence[position]:
               neighbor[position]=aa
               neighbor_string="".join(neighbor)
               if sequence_fitness<model.get_fitness(neighbor_string):
                  return 0
               neighbor=[s for s in sequence]
    return 1

In [ ]:
from models.Ground_truth_oracles.TF_binding_landscape_models import TF_binding_landscape

In [ ]:
model=TF_binding_landscape()

In [ ]:
model.construct(landscapes[10])

In [ ]:
len(model.sequences)

In [ ]:
all_seqs={}
for landscape in landscapes:
    model=TF_binding_landscape()
    model.construct(landscape)
    max_peak=-1
    current_max="AAAAAAAA"
    for seq in model.sequences:
        if not seq in all_seqs:
           all_seqs[seq]=[0,[]]
        if is_sequence_a_peak(model,seq):
           all_seqs[seq][0]+=1
           if model.get_fitness(seq)>max_peak:
              max_peak = model.get_fitness(seq)
              current_max= seq
    all_seqs[current_max][1].append(landscape[landscape.index("data/TF_binding_landscapes/"):landscape.index(".txt")])
        

In [ ]:
count=0

for k in sorted(all_seqs, key=all_seqs.get, reverse=False):
    count+=1
    print(k, all_seqs[k])
    if count>10:
        break

In [ ]:
count=0

for k in sorted(all_seqs, key=all_seqs.get, reverse=True):
    count+=1
    print(k, all_seqs[k])
    if count>10:
        break

In [ ]:
selected_sequences=[ 'TTAATTAA',   #  often global peak
                     'GCTCGAGC','GCGCGCGC','TGCGCGCC','ATATAGCC','GTTTGGTA',"ATTATGTT",  #  often local peaks
                    "CAGTTTTT","AAAAATTT", "AAAAACGC", "GTTGTTTT", "TGCTTTTT","AAAGATAG","CCTTCTTT","AAAGAGAG"] # never local peaks 

In [ ]:
with open("TF_starting_id.csv","w") as outfile:
    count=0
    outfile.write("id,start,local_peak_in_num_landscapes,global_peak_in_landscapes\n")
    
    for seq in selected_sequences:
        if len(all_seqs[seq][1])>0:
            last_col=";".join([x.strip("data/TF_binding_landscapes/").strip("_8mers") for x in all_seqs[seq][1]])
        else:
            last_col=""
        outfile.write(f'TF{count},{seq},{all_seqs[seq][0]},{last_col}\n')
        count+=1